Merge gene data from all rows of GWAS and ClinVar tables in order to start the analysis on the differential expression. 

In [1]:
# imports
import pandas as pd
import openpyxl
import re

In [2]:
# load clinvar data

clinvar = pd.read_excel('/Users/gilsasson/Data Science Project/clinvar_associations_melted_by_trait.xlsx', sheet_name="Sheet1")
print(clinvar)
clinvar_dict = dict()

for _, row in clinvar.iterrows():
    key = row.iloc[1]       # your trait column
    raw   = row.iloc[2]     # the pipe‐delimited gene string

    # split & strip into a flat list
    genes = [g.strip()
             for g in str(raw).split("|")
             if g.strip()]

    # accumulate into the dict
    clinvar_dict.setdefault(key, []).extend(genes)

# finally, dedupe + sort each list
for trait, gene_list in clinvar_dict.items():
    clinvar_dict[trait] = sorted(set(gene_list))

clinvar_dict.items()

    Unnamed: 0            Human disease name  \
0            0                     alzheimer   
1            1                     aneurysms   
2            2                     carcinoma   
3            3                          gout   
4            4               inguinal hernia   
5            5            muscular dystrophy   
6            6         myocardial infarction   
7            7                osteoarthritis   
8            8                  osteoporosis   
9            9                  preeclampsia   
10          10                     psoriasis   
11          11          rheumatoid arthritis   
12          12                 schizophrenia   
13          13                   sleep apnea   
14          14  sudden infant death syndrome   

                                            gene_name  
0   APP|PSEN1|GRN|VCP|MAPT|CSF1R|MPO|ABCA7|SORCS1|...  
1                                                FBN1  
2   MSH3|RASA1|MSH6|BARD1|HNF1B|NBN|PTCH1|CDH1|ATM...  
3   RNA

dict_items([('alzheimer', ['ABCA7', 'APP', 'CSF1R', 'GRN', 'HFE', 'MAPT', 'MPO', 'PSEN1', 'PSEN2', 'SORCS1', 'VCP']), ('aneurysms', ['FBN1']), ('carcinoma', ['ABL1', 'ACVR1', 'AKT1', 'ALK', 'APC', 'ARAF', 'ARID1B', 'ARID2', 'ATM', 'ATRX', 'AXIN2', 'B2M', 'BARD1', 'BCOR', 'BRAF', 'BRCA1', 'BRCA2', 'BRIP1', 'CDC73', 'CDH1', 'CDK12', 'CDK4', 'CDKN2A', 'CHEK2', 'CNOT9', 'CREBBP', 'CRNKL1', 'CTHRC1', 'CTNNA1', 'CTNNB1', 'CYP21A2', 'DCC', 'DDR2', 'DLC1', 'DNMT3A', 'EGFR', 'ELOC', 'EP300', 'EPHA2', 'EPHA5', 'ERBB2', 'ERBB3', 'FANCG', 'FANCM', 'FAS', 'FBXW7', 'FGF10', 'FGFR1', 'FGFR2', 'FGFR3', 'FLCN', 'GNAS', 'H3-3A', 'H3C11', 'HNF1B', 'HOXB13', 'HRAS', 'IDH1', 'IDH2', 'IGF2R', 'ING1', 'IRF1', 'IRF3', 'JAK3', 'KIT', 'KNSTRN', 'KRAS', 'LIG4', 'LZTS1', 'MAP2K1', 'MAP2K2', 'MAPK1', 'MCC', 'MED12', 'MET', 'MLH1', 'MRE11', 'MSH2', 'MSH3', 'MSH6', 'MSR1', 'MTOR', 'MUTYH', 'MYC', 'MYCN', 'NBN', 'NDUFA13', 'NFE2L2', 'NOTCH1', 'NRAS', 'NTRK1', 'PALB2', 'PDGFRL', 'PIK3CA', 'PIK3R2', 'PMS2', 'POLE', 'PP

In [3]:
import ast
# load gwas data
gwas = pd.read_pickle(
    '/Users/gilsasson/Data Science Project/GWAS/traits.pkl'
)

# 4) Now coerce your numeric-looking lists into lists of float
for col in ['prop_studies', 'OR', 'BETA']:
    gwas[col] = gwas[col].apply(lambda lst: [float(x) for x in lst])

# 5) Build your dicts
mapped_dict   = {}

for disease, sub in gwas.groupby("Parent_trait"):
    # flatten each column in lockstep
    mapped_flat = [g for lst in sub['MAPPED_GENE']         for g in lst]
    counts_flat = [c for lst in sub['prop_studies']   for c in lst]
    or_flat     = [o for lst in sub['OR']                  for o in lst]
    beta_flat   = [b for lst in sub['BETA']                for b in lst]

    # sanity check
    if not (len(mapped_flat) == len(counts_flat) == len(or_flat) == len(beta_flat)):
        raise ValueError(
            f"Length mismatch in {disease}: "
            f"{len(mapped_flat)}, {len(counts_flat)}, "
            f"{len(or_flat)}, {len(beta_flat)}"
        )

    # store: mapped_dict → (genes, counts, ORs, BETAs)
    mapped_dict[disease] = (
        sorted(set(mapped_flat)),
        counts_flat,
        or_flat,
        beta_flat
    )

# 6) Inspect
print("Mapped genes for iron deficiency:",   mapped_dict['iron deficiency'][0])
print("Counts for iron deficiency:",         mapped_dict['iron deficiency'][1])
print("OR for iron deficiency:",             mapped_dict['iron deficiency'][2])
print("BETA for iron deficiency:",           mapped_dict['iron deficiency'][3])


Mapped genes for iron deficiency: ['ABHD17AP5 - IGLV4-60']
Counts for iron deficiency: [1.0]
OR for iron deficiency: [0.0]
BETA for iron deficiency: [0.0]


In [4]:
# merge clinvar and gwas data:
disease_names = {"Alzheimer's Disease": ["Alzheimer_s disease", "Alzheimer_s disease 2", "Alzheimer_s disease 3", "Alzheimer_s disease 4", "alzheimer"],
                 "Acne vulgaris": ["Acne vulgaris"], "aneurysm": ["aortic aneurysm", "brain aneurysm", "aneurysms"],
                 "Back Disorders": ["Back Disorders 1", "Back Disorders 2"], "Bipolar disorder": ["Bipolar disorder"], "Bronchial asthma": ["Bronchial asthma"],
                 "carcinoma": ["carcinoma"], "Chronic sinusitis": ["Chronic sinusitis"], "Colon diverticulosis": ["Colon diverticulosis", "Colon diverticulosis (better)"],
                 "Endometriosis": ["Endometriosis"], "Frailty": ["Frailty"], "Gout": ["Gout", "gout"], "Hemorrhoids": ["Hemorrhoids"],
                 "Inguinal hernia": ["Inguinal hernia", "inguinal hernia"], "iron deficiency": ["iron deficiency"], "male infertility": ["male infertility"],
                 "Morning sickness": ["Morning sickness"], "Muscular dystrophy": ["Muscular dystrophy", "muscular dystrophy"], "myocardial infraction": ["myocardial infraction"],
                 "Obstetric and Prenatal Difficulties": ["Obstetric and Prenatal Difficulties"], "Osteoarthritis": ["Osteoarthritis", "osteoarthritis"],
                 "Osteoporosis": ["Osteoporosis", "osteoporosis"], "Ovarian polycystic syndrome (PCOS)": ["Ovarian polycystic syndrome (PCOS)"],
                 "Preeclampsia": ["Preeclampsia with parent trait", "preeclampsia"], "premature birth": ["premature birth"], "Psoriasis": ["Psoriasis", "psoriasis"],
                 "Schizophrenia": ["Schizophrenia", "schizophrenia"], "rheumatoid arthritis": ["rheumatoid arthritis"], "Scoliosis": ["Scoliosis"],
                 "Sleep apnea": ["Sleep apnea", "sleep apnea"], "Varicose veins": ["Varicose veins"], "sudden infant death syndrome": ["sudden infant death syndrome"]}

joint_clinvar_dict  = {}


In [5]:
# for each trait, flatten into a long DataFrame
long_rows = []
for std_trait, syns in disease_names.items():
    cset = set()
    for syn in syns:

        # clinvar_dict likewise disease→[genes]
        cset.update(clinvar_dict.get(syn, []))
        joint_clinvar_dict [std_trait] = sorted(cset)
        # now, for gwas
        genes, counts, ORs, BETAs = mapped_dict.get(syn, ([], [], [], []))
        # zip them into long rows
        for g, c, o, b in zip(genes, counts, ORs, BETAs):
            long_rows.append({
                'Trait':            std_trait,
                'gene':             g,
                'normalized_count': c,
                'OR':                o,
                'BETA':              b
            })

long_df = pd.DataFrame(long_rows)

# group by trait & gene, taking means
mean_df = (
    long_df
    .groupby(['Trait','gene'], as_index=False)
    .agg({
        'normalized_count':'mean',
        'OR':               'mean',
        'BETA':             'mean'
    })
)

# for each trait, collect lists of gene & mean‐values
gwas_mean_data = {}
for trait, sub in mean_df.groupby('Trait'):
    # these are parallel lists, guaranteed aligned
    genes   = sub['gene'].tolist()
    counts  = sub['normalized_count'].tolist()
    ORs     = sub['OR'].tolist()
    BETAs   = sub['BETA'].tolist()
    gwas_mean_data[trait] = (genes, counts, ORs, BETAs)

# build df_wide
rows = []
for trait in disease_names:
    genes, counts, ORs, BETAs = gwas_mean_data.get(trait, ([],[],[],[]))
    clinvar = joint_clinvar_dict.get(trait, [])
    rows.append({
        'Disease':               trait,
        'Mapped_GWAS_Genes':     genes,
        'Mapped_GWAS_MeanCounts':counts,
        'Mapped_GWAS_MeanOR':     ORs,
        'Mapped_GWAS_MeanBETA':   BETAs,
        'ClinVar_Genes':         clinvar,
        'All_Genes':           set(genes + clinvar),
        'N_Mapped':              len(genes),
        'N_ClinVar':             len(clinvar),
        'N_All':                 len(set(genes + clinvar))
    })

df_wide = pd.DataFrame(rows)
df_wide.head()


,Disease,Mapped_GWAS_Genes,Mapped_GWAS_MeanCounts,Mapped_GWAS_MeanOR,Mapped_GWAS_MeanBETA,ClinVar_Genes,All_Genes,N_Mapped,N_ClinVar,N_All
0,Alzheimer's Disease,"[AADACL4 - AADACL3, AAR2 - DLGAP4, AARS2 - SPA...","[0.0784313725490196, 0.058823529411764705, 0.1...","[0.0, 0.0, 2.277689833333333, 0.0, 0.0, 0.0, 0...","[0.0, 0.17193333333333335, 0.0, 0.10865, 0.065...","[ABCA7, APP, CSF1R, GRN, HFE, MAPT, MPO, PSEN1...","{HLA-DRB1, PSMC3 - RAPSN, AFG1L - FOXO3, MCTP1...",2214,11,2222
1,Acne vulgaris,"[ADAMTS18 - LINC02131, ADORA2A-AS1, AFG2A, AFG...","[0.4, 0.4, 0.4, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2, ...","[1.295, 0.0, 0.0, 0.0, 11.67, 3.46, 2.17, 1.63...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],"{UPB1, SDK1, PINX1, PINX1, LAMC2, RPSAP72 - TE...",73,0,73
2,aneurysm,"[ADAMTS8, ZBTB44-DT, ALDH2, ANGPT4, ANKRD44, A...","[0.25, 0.75, 0.375, 0.25, 0.25, 0.25, 0.25, 0....","[1.28, 1.3026000000000002, 1.315, 1.1905000000...","[0.0, 0.1821333333333333, 0.15295, 0.109855000...",[FBN1],"{NAPA-AS1, BOLL, HDAC9, ALDH2, FERD3L - POLR1F...",113,1,113
3,Back Disorders,"[ACTA2, FAS, ADAM7-AS1, ADCY3, ADCY7, ADGRL2, ...","[0.4, 0.4, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, ...","[1.2974999999999999, 0.0, 1.72, 0.0, 1.2349999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],"{MIR4302 - RASSF8-AS1, UBASH3A, HMGN2P18 - KRT...",316,0,316
4,Bipolar disorder,"[ACADS - SPPL3, ACTG1P22 - VRK2, ACTN3, ACTR5 ...","[0.16666666666666666, 0.14814814814814814, 0.1...","[1.1910699999999999, 1.1162714285714286, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],"{TFEC, BABAM2, MRPL33, CAMK1D, MYOSLID-AS1, KI...",940,0,940


In [7]:
# save the data to an excel file
df_wide.to_csv('/Users/gilsasson/PycharmProjects/pythonProject/ds_project/joint_gene_list_per_trait.csv', index=False)